In [1]:
import json
import numpy as np
import pandas as pd
from scipy import linalg
from sklearn.feature_extraction.text import CountVectorizer

## Input:

1. journal_entry → string


2. parameters → .npz file containing:
    * U → NumPy matrix (int)
    * sigma → NumPy matrix (int)
    * VT → NumPy matrix (int)
    * vocab → NumPy matrix (str)


3. tasks_json
    * keys are just indices
    * example:
    
>```
{
	"0": "Did you know: It's better to challenge a fear food...",
	"1": "Did you know: When you eat low-carbs during the day...",
	"2": "Did you know: Asians eat lots of rice and carbs...",
	"3": "Did you know: If you start eating when you're not hungry...",
	"4": "Did you know: If you don't have enough carbs...",
	"5": "Did you know: Fibre helps digestion and is necessary...",
	"6": "Did you know: It is recommended that people consume fish...",
	"7": "Did you know: Many people consume far more protein than they need..."
}```

In [1]:
def nlp(journal_entry,parameters,tasks_json,dim=100000):
    
    npzfile = np.load(parameters)
    U, sigma, VT, vocab = npzfile['U'], npzfile['sigma'], npzfile['VT'], npzfile['vocab']
    
    U = U[:,:dim+1]
    sigma = sigma[:dim+1,:dim+1]
    VT = VT[:dim+1,:]
    
    cv = CountVectorizer(vocabulary=vocab)
    query = np.matrix(cv.fit_transform([journal_entry]).toarray())
    
    query_proj = query*U*np.linalg.inv(sigma)
    v = query_proj.tolist()[0]

    similarity = []
    for coords in VT.T:
        u = coords
        similarity.extend([np.dot(u,v)/np.linalg.norm(u)/np.linalg.norm(v)])
    
    with open(tasks_json, 'r') as fp:
        tasks_dict = json.load(fp)
    indices = list(tasks_dict.keys())
    tasks = list(tasks_dict.values())
    sim, i, recs = zip(*sorted(zip(similarity, indices, tasks), key=lambda x: x[0], reverse=True))
    
    recs_dict = dict(zip(list(i),list(recs)))
    
    return(recs_dict)



## Output:

1. recommendations → json
    * essentially mimics the `tasks_json` input, but sorted
    * no new indices, just sorted
    * example:

>```
{
    "2": "Did you know: Asians eat lots of rice and carbs...",
    "1": "Did you know: When you eat low-carbs during the day...",
    "3": "Did you know: If you start eating when you’re not hungry...",
    "5": "Did you know: Fibre helps digestion and is necessary...",
    "4": "Did you know: If you don’t have enough carbs...",
    "0": "Did you know: It’s better to challenge a fear food...",
    "7": "Did you know: Many people consume far more protein than they need...",
    "6": "Did you know: It is recommended that people consume fish.."
}```